# Request VC from Authority

This notebook is used in combination with notebook `01_issue_VC_city.ipynb` (see Authority agent). Break points indicate when to switch notebooks.

In [1]:
%%javascript
document.title ='Manufacturer1 Agent'

<IPython.core.display.Javascript object>

#### Imports

In [2]:
%autoawait 
from aries_cloudcontroller import AriesAgentController
from libs.connection_service import ConnectionService
import libs.functions as fnc
import os
from pprintpp import pprint
from termcolor import colored

IPython autoawait is `on`, and set to use `asyncio`


#### Initialize `Manufacturer1` Agent Controller

In [3]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://manufacturer1-agent:3021 and an api key of adminApiKey


## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [4]:
# Setup
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


In [5]:
connections = ConnectionService(agent_controller, role="prover")

Init ConnectionService:
* Defines and registers agent listeners for prover:  ['connections', 'basicmessages', 'present_proof']
* Stores initiated connections
* Allows to easily create and accept connection invitations
* Facilitates process of issuing, verifying, or proving verifiable credentials


In [6]:
# Check if Manufacturer already has a VC
connections.get_credentials()

{'results': []}

#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

#### 1.3 – Register listeners with `agent_controller`

## 2 – Establish a connection with Authority agent
A connection with the credential issuer (i.e., the authority agent) must be establieshed before a VC can be received. In this scenario, the agent requests a connection with the Authority to be certified as an official city agency. Thus, the city agent sends an invitation to the Authority.

### 2.1 Create invitation to Authority agent

In [7]:
# Setup for connection with Authority agent
alias = None
auto_accept = "true" # Accept response of Authority agent right away
public = "false" # Do not use public DID
multi_use = "false" # Invitation is only for one invitee

connection_id = connections.create_connection_invitation(alias=alias, auto_accept=auto_accept, public=public, multi_use=multi_use)


---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  0801e8a4-fbe7-412b-9b09-cf553c1d8686
State :  invitation (invitation-sent)
Routing State : none
Their Role :  invitee
---------------------------------------------------------------------

Copy & paste invitation and share with external agent:
{
    '@id': '68c1a20a-bea6-4e7b-9e49-58b845200170',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'Manufacturer1',
    'recipientKeys': ['7Zhz1XoAWj4nmZUTw6PyjhAcsN3Eqk6JRisaA6TkQqrY'],
    'serviceEndpoint': 'https://e2120909fb85.ngrok.io',
}



---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  0801e8a4-fbe7-412b-9b09-cf553c1d8686
State :  request (request-received)
Routing State : none
Connection with :  Authority
Their Role :  invitee
---------------------------------------------------------------------


Send trust ping to finalize connection? [yes/no] yes



---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  0801e8a4-fbe7-412b-9b09-cf553c1d8686
State :  response (response-sent)
Routing State : none
Connection with :  Authority
Their Role :  invitee
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  0801e8a4-fbe7-412b-9b09-cf553c1d8686
State :  active (completed)
Routing State : none
Connection with :  Authority
Their Role :  invitee
---------------------------------------------------------------------
Connection ID: 0801e8a4-fbe7-412b-9b09-cf553c1d8686 is now active.


**BREAK POINT:** Please switch to agent `Authority`, open `01_issue_VC_city.ipynb`, and continue with Step 2

---

### 2.3 – Sent trust ping to establish and activate connection

## 3 – Request VC from `Authority` agent
### 3.1 – Message `Authority` to request a VC

In [ ]:
connections.send_message(connection_id, "testing connection")

In [ ]:
basic_message = "Hello Authority agent"
await agent_controller.messaging.send_message(connection_id, basic_message)

**BREAK POINT:** Go to Step 3 in the `01_issue_VC_city.ipynb` notebook of the `Authority` agent.

---

### 3.1 – Provide `Authority` with the relevant information to issue a VC

In [ ]:
basic_message = '{"manufacturerName": "undisclosedName1", "manufacturerCountry": "DE", "manufacturerCity": "Munich"}'
await agent_controller.messaging.send_message(connection_id, basic_message)

**BREAK POINT:** Return to step 

---


### 3.2 – Request VC from `Authority`'s Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.

In [8]:
identifiers = fnc.get_identifiers()
schema_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
schema_id

'PGtDVMGGbrfYNFjcN85Sxk:2:certify-manufacturer:0.0.1'

In [9]:
connections.request_vc(connection_id, schema_id)

Found record for PGtDVMGGbrfYNFjcN85Sxk:2:certify-manufacturer:0.0.1 at state offer_received
await_record:
{
    'auto_issue': False,
    'auto_offer': False,
    'auto_remove': False,
    'connection_id': '0801e8a4-fbe7-412b-9b09-cf553c1d8686',
    'created_at': '2021-08-12 07:54:21.966606Z',
    'credential_definition_id': 'PGtDVMGGbrfYNFjcN85Sxk:3:CL:248873:default',
    'credential_exchange_id': 'efd23003-b6a0-4dd7-a49e-e63700f8ddf4',
    'credential_offer': {
        'cred_def_id': 'PGtDVMGGbrfYNFjcN85Sxk:3:CL:248873:default',
        'key_correctness_proof': {
            'c': '37154411101356200017909389431505652102339810726196636894041130256688831214919',
            'xr_cap': [
                [
                    'manufacturername',
                    '7165563436195852115905746733439192526534090111864222258013457570645438960379727538570671719023450439143543407188760655060411252654442135489935531596539324690301158998830988562128908439039736810368494151021009674087427951553379

Do you want to store the VC with ID efd23003-b6a0-4dd7-a49e-e63700f8ddf4 [yes/no] y
Please provide a Credential ID for VC with Record ID efd23003-b6a0-4dd7-a49e-e63700f8ddf4 M1-isManufacturer-VC


store_cred_response
{
    'auto_issue': False,
    'auto_offer': False,
    'auto_remove': False,
    'connection_id': '0801e8a4-fbe7-412b-9b09-cf553c1d8686',
    'created_at': '2021-08-12 07:54:21.966606Z',
    'credential': {
        'attrs': {
            'isManufacturer': 'TRUE',
            'manufacturerCity': 'Berlin',
            'manufacturerCountry': 'Germany',
            'manufacturerName': 'undisclosedManufacturer1',
        },
        'cred_def_id': 'PGtDVMGGbrfYNFjcN85Sxk:3:CL:248873:default',
        'referent': 'M1-isManufacturer-VC',
        'schema_id': 'PGtDVMGGbrfYNFjcN85Sxk:2:certify-manufacturer:0.0.1',
    },
    'credential_definition_id': 'PGtDVMGGbrfYNFjcN85Sxk:3:CL:248873:default',
    'credential_exchange_id': 'efd23003-b6a0-4dd7-a49e-e63700f8ddf4',
    'credential_id': 'M1-isManufacturer-VC',
    'credential_offer': {
        'cred_def_id': 'PGtDVMGGbrfYNFjcN85Sxk:3:CL:248873:default',
        'key_correctness_proof': {
            'c': '3715441110135620001

### 3.3 – Store received VC in wallet

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [13]:
connections.get_credentials()

{'results': [{'referent': 'M1-isManufacturer-VC',
   'attrs': {'manufacturerCity': 'Berlin',
    'manufacturerCountry': 'Germany',
    'manufacturerName': 'undisclosedManufacturer1',
    'isManufacturer': 'TRUE'},
   'schema_id': 'PGtDVMGGbrfYNFjcN85Sxk:2:certify-manufacturer:0.0.1',
   'cred_def_id': 'PGtDVMGGbrfYNFjcN85Sxk:3:CL:248873:default',
   'rev_reg_id': None,
   'cred_rev_id': None}]}

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [14]:
await agent_controller.terminate()